In [1]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import kagglehub

# Download the dataset
path = kagglehub.dataset_download("borhanitrash/animal-image-classification-dataset")
dataSetPath = path + "/Animals"

# Image size & batch size
imageSize = (256, 256)
batchSize = 32

# Load dataset with automatic labels
dataSet = tf.keras.preprocessing.image_dataset_from_directory(
    dataSetPath,
    image_size = imageSize,
    batch_size = batchSize,
    validation_split = 0.15,  # Split into 80% training, 20% validation
    subset="training",
    seed=42
)

valSet = tf.keras.preprocessing.image_dataset_from_directory(
    dataSetPath,
    image_size=imageSize,
    batch_size=batchSize,
    validation_split=0.15,
    subset="validation",
    seed=42
)

# Class names (cats, dogs, snakes)
classNames = dataSet.class_names
numClasses = len(classNames)
print("Class names:", classNames)

# Normalize the data (Scaling between 0 and 1)
normalization_layer = tf.keras.layers.Rescaling(1./255)
dataSet = dataSet.map(lambda x, y: (normalization_layer(x), y))
valSet = valSet.map(lambda x, y: (normalization_layer(x), y))


# Single Layer Perceptron
SLPModel = keras.Sequential([
    keras.layers.Flatten(input_shape=(256, 256, 3)),  # Flatten input
    keras.layers.Dense(numClasses, activation='softmax')  # Output layer with softmax
])


optimizer = keras.optimizers.Adam(learning_rate=0.000001)
SLPModel.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

SLPModel.fit(dataSet, epochs=30, validation_data=valSet)

_, accuracy_slp = SLPModel.evaluate(valSet, verbose=0)
print('Single Layer Perceptron Accuracy: %.2f' % (accuracy_slp * 100))

# Multilayer Perceptron
MLPModel = keras.Sequential([
    keras.layers.Flatten(input_shape=(256, 256, 3)),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(numClasses, activation='softmax')
])


optimizer = keras.optimizers.Adam(learning_rate=0.000001)
MLPModel.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

MLPModel.fit(dataSet, epochs=15, validation_data=valSet)

_, accuracy_mlp = MLPModel.evaluate(valSet, verbose=0)
print('Multilayer Perceptron Accuracy: %.2f' % (accuracy_mlp * 100))

# Predictions
slpPredictedY = np.argmax(SLPModel.predict(valSet), axis=-1)
mlpPredictedY = np.argmax(MLPModel.predict(valSet), axis=-1)

# Extract true labels from the validation dataset
actualY = np.concatenate([y.numpy() for x, y in valSet], axis=0)

# Calculate accuracy using scikit-learn
accuracy_slp_sklearn = accuracy_score(actualY, slpPredictedY)
accuracy_mlp_sklearn = accuracy_score(actualY, mlpPredictedY)

print("Single Layer Perceptron Accuracy:", accuracy_slp_sklearn)
print("Multilayer Perceptron Accuracy:", accuracy_mlp_sklearn)

100%|██████████| 38.4M/38.4M [00:02<00:00, 18.4MB/s]

Extracting files...


Found 3000 files belonging to 3 classes.
Using 2550 files for training.
Found 3000 files belonging to 3 classes.
Using 450 files for validation.
Class names: ['cats', 'dogs', 'snakes']
Epoch 1/30


/usr/local/lib/python3.11/dist-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 61ms/step - accuracy: 0.3586 - loss: 1.1818 - val_accuracy: 0.3511 - val_loss: 1.1078
Epoch 2/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 76ms/step - accuracy: 0.3704 - loss: 1.1017 - val_accuracy: 0.3956 - val_loss: 1.0856
Epoch 3/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.3938 - loss: 1.0811 - val_accuracy: 0.4089 - val_loss: 1.0720
Epoch 4/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.4013 - loss: 1.0716 - val_accuracy: 0.4422 - val_loss: 1.0580
Epoch 5/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 6s 71ms/step - accuracy: 0.4285 - loss: 1.0627 - val_accuracy: 0.4533 - val_loss: 1.0523
Epoch 6/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.4341 - loss: 1.0531 - val_accuracy: 0.4511 - val_loss: 1.0463
Epoch 7/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 5s 67ms/step - accuracy: 0.4492 - loss: 1.0483 - val_accuracy: 0.4667 - val_loss: 1.0468
Epoch 8/30
80/80 ━━━━━━━━━━━━━━━━━━━━ 9s 57ms/step - accuracy: 0.4640 - loss: 1.0436 - val_accuracy: 0.4622 - val_loss: 1.

In [4]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from sklearn.metrics import accuracy_score
import numpy as np
import kagglehub

# Enable Eager Execution
tf.config.run_functions_eagerly(True)

# Download dataset
path = kagglehub.dataset_download("borhanitrash/animal-image-classification-dataset")
dataSetPath = path + "/Animals"

# Image size & batch size
imageSize = (128, 128)  # Reduced size for efficiency
batchSize = 32

# Load dataset with automatic labels
dataSet = tf.keras.preprocessing.image_dataset_from_directory(
    dataSetPath,
    image_size=imageSize,
    batch_size=batchSize,
    validation_split=0.15,
    subset="training",
    seed=42
)

valSet = tf.keras.preprocessing.image_dataset_from_directory(
    dataSetPath,
    image_size=imageSize,
    batch_size=batchSize,
    validation_split=0.15,
    subset="validation",
    seed=42
)

# Class names
classNames = dataSet.class_names
numClasses = len(classNames)
print("Class names:", classNames)

# Data Augmentation
dataAugmentation = keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
])

# Normalize data
normalization_layer = tf.keras.layers.Rescaling(1./255)
dataSet = dataSet.map(lambda x, y: (dataAugmentation(normalization_layer(x)), y))
valSet = valSet.map(lambda x, y: (normalization_layer(x), y))

# Ensure dataset is preloaded into memory
dataSet = dataSet.prefetch(buffer_size=tf.data.AUTOTUNE)
valSet = valSet.prefetch(buffer_size=tf.data.AUTOTUNE)

# Single Layer Perceptron Model
SLPModel = keras.Sequential([
    keras.layers.Flatten(input_shape=(128, 128, 3)),
    keras.layers.Dense(numClasses, activation='softmax')
])

optimizer_slp = keras.optimizers.Adam(learning_rate=0.001)
SLPModel.compile(optimizer=optimizer_slp, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
SLPModel.fit(dataSet, epochs=20, validation_data=valSet)

# Multilayer Perceptron Model
MLPModel = keras.Sequential([
    keras.layers.Flatten(input_shape=(128, 128, 3)),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(numClasses, activation='softmax')
])

optimizer_mlp = keras.optimizers.Adam(learning_rate=0.001)
MLPModel.compile(optimizer=optimizer_mlp, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
MLPModel.fit(dataSet, epochs=15, validation_data=valSet)

# CNN Transfer Learning (MobileNetV2)
baseModel = keras.applications.MobileNetV2(input_shape=(128, 128, 3), include_top=False, weights="imagenet")
baseModel.trainable = False  # Freeze pre-trained layers

CNNModel = keras.Sequential([
    baseModel,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.3),
    layers.Dense(numClasses, activation='softmax')
])

optimizer_cnn = keras.optimizers.Adam(learning_rate=0.001)
CNNModel.compile(optimizer=optimizer_cnn, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
CNNModel.fit(dataSet, epochs=10, validation_data=valSet)

# Evaluate Models
_, accuracy_slp = SLPModel.evaluate(valSet, verbose=0)
_, accuracy_mlp = MLPModel.evaluate(valSet, verbose=0)
_, accuracy_cnn = CNNModel.evaluate(valSet, verbose=0)
print(f'SLP Accuracy: {accuracy_slp:.2%}')
print(f'MLP Accuracy: {accuracy_mlp:.2%}')
print(f'CNN Accuracy: {accuracy_cnn:.2%}')

# Predictions
slpPredictedY = np.argmax(SLPModel.predict(valSet), axis=-1)
mlpPredictedY = np.argmax(MLPModel.predict(valSet), axis=-1)
cnnPredictedY = np.argmax(CNNModel.predict(valSet), axis=-1)

# Extract true labels
actualY = np.concatenate([y.numpy() for _, y in valSet], axis=0)

# Accuracy using scikit-learn
print("SLP Accuracy (sklearn):", accuracy_score(actualY, slpPredictedY))
print("MLP Accuracy (sklearn):", accuracy_score(actualY, mlpPredictedY))
print("CNN Accuracy (sklearn):", accuracy_score(actualY, cnnPredictedY))


Found 3000 files belonging to 3 classes.
Using 2550 files for training.
Found 3000 files belonging to 3 classes.
Using 450 files for validation.
Class names: ['cats', 'dogs', 'snakes']
Epoch 1/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 15s 189ms/step - accuracy: 0.3330 - loss: 6.1585 - val_accuracy: 0.3200 - val_loss: 4.7412
Epoch 2/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 19s 175ms/step - accuracy: 0.3674 - loss: 2.4427 - val_accuracy: 0.3578 - val_loss: 5.4292
Epoch 3/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 15s 182ms/step - accuracy: 0.3856 - loss: 3.8415 - val_accuracy: 0.3956 - val_loss: 2.8781
Epoch 4/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 14s 178ms/step - accuracy: 0.3865 - loss: 2.3004 - val_accuracy: 0.4133 - val_loss: 1.7615
Epoch 5/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 20s 176ms/step - accuracy: 0.4044 - loss: 2.2521 - val_accuracy: 0.3800 - val_loss: 2.3574
Epoch 6/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 14s 175ms/step - accuracy: 0.4029 - loss: 2.7276 - val_accuracy: 0.4178 - val_loss: 1.6272
Epoch 7/20
80/80 ━━━━━━━━━━━━━━━━━━━━ 21s